In [15]:
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import CheckpointCallback

from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise
from lib.environment.attacker_v2.environment import Environment

import numpy as np
import os
from datetime import datetime

from lib.helpers.configuration_helper import ConfigurationHelper

In [16]:
task_training_name = "attacker_v2"

In [17]:
def create_env():
    def _init():
        return Environment()
    return _init

In [18]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [19]:
def get_task_models_path():
    current_datetime = datetime.now()

    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    minute = current_datetime.minute
    second = current_datetime.second

    datetime_name = f"{year}_{month}_{day}_{hour}_{minute}_{second}"

    return f"models/{task_training_name}/{datetime_name}"

In [20]:
save_path = get_task_models_path()

create_folder_if_not_exists(save_path)

In [21]:
num_threads = 8
model_filename_prefix = "DDPG_model"

In [22]:
env = SubprocVecEnv([create_env() for i in range(num_threads)])

n_actions = env.action_space.shape[-1]
n_actions

2024-05-17 15:04:01.474679: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-17 15:04:01.529707: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-17 15:04:01.565448: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 15:04:01.588724: I tensorflow/core/util/port.cc:113] oneDNN custom op

2

In [23]:
time_step = ConfigurationHelper.getTrainingTimeStep()

In [24]:
action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=.1 * np.ones(n_actions), dt=time_step)

In [25]:
tau = 0.005
gamma = 0.99
learning_rate = 0.001
policy = "MlpPolicy"

In [26]:
load_model = True
loaded_model_path = "models/attacker_v2/2024_5_15_12_10_33/DDPG_model_20000000_steps"

In [27]:
model = DDPG(
        policy=policy,
        env=env,
        tau=tau,
        gamma=gamma,
        action_noise=action_noise,
        learning_rate=learning_rate,
        verbose=2)

if load_model:
    model.set_parameters(loaded_model_path)

Using cpu device


In [28]:
total_timesteps = 10000000

In [29]:
saved_model_number = 10
save_freq = total_timesteps // (saved_model_number * num_threads)
log_interval = total_timesteps // 10

In [30]:
checkpoint_callback = CheckpointCallback(
    save_freq=save_freq,
    save_path=save_path,
    name_prefix=model_filename_prefix)

In [31]:
model.learn(
    total_timesteps=total_timesteps,
    log_interval=log_interval,
    callback=checkpoint_callback,
    progress_bar=True)

Output()


ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)


...


ODE Message 3: LCP internal error, s <= 0 (s=0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)



ODE Message 3: LCP internal error, s <= 0 (s=-0.0000e+00)


In [32]:
model.save(f"{save_path}/DDPG_model")